In [39]:
import pandas as pd
import numpy as np
from pathlib import Path

In [40]:
pd.set_option("display.max_columns", None)

In [41]:
METHODS = ("qsort_c", "vanilla_quicksort", "insertion_sort")
INPUT_TYPES = ("reverse_sorted", "single_num", "sorted", "uniform", "unsorted", "N/A")

IN_FILE = Path("./results/2021-07-07_09-52-14/output_2021-07-07_09-52-02.csv")
df = pd.read_csv(IN_FILE)

# Convert microseconds to milliseconds and rename column
df["Elapsed Time (microseconds)"] = df["Elapsed Time (microseconds)"] / 1000
df.rename(columns={"Elapsed Time (microseconds)": "Runtime (milliseconds)"}, inplace=True)

# Add seconds column
df["Runtime (seconds)"] = df["Runtime (milliseconds)"] / 1000

# Reorder columns
df = df[["Input", "Description", "Method", "Size", "Threshold", "Runtime (milliseconds)", "Runtime (seconds)", "Valid"]]


In [42]:
df

,Input,Description,Method,Size,Threshold,Runtime (milliseconds),Runtime (seconds),Valid
0,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,single_num,qsort_c,150000,18,18.733,0.018733,True
1,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,single_num,qsort_c,100000,3,14.831,0.014831,True
2,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,single_num,qsort_c,100000,27,11.618,0.011618,True
3,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,single_num,qsort_c,150000,46,18.656,0.018656,True
4,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,single_num,qsort_c,150000,8,20.145,0.020145,True
...,...,...,...,...,...,...,...,...
4935,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,reverse_sorted,insertion_sort,900000,1,701579.132,701.579132,True
4936,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,reverse_sorted,insertion_sort,950000,1,781818.180,781.818180,True
4937,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,single_num,vanilla_quicksort,800000,1,1252216.586,1252.216586,True
4938,/pfs/tc1/project/mallet/jarulsam/quicksort-tun...,single_num,vanilla_quicksort,850000,1,1422265.793,1422.265793,True


In [43]:
df.groupby(["Method", "Description", "Size"]).mean()

Threshold  Runtime (milliseconds)  \
Method            Description    Size                                        
insertion_sort    reverse_sorted 50000         1.0                2150.072   
                                 100000        1.0                8632.831   
                                 150000        1.0               18871.199   
                                 200000        1.0               33562.805   
                                 250000        1.0               54213.657   
...                                            ...                     ...   
vanilla_quicksort unsorted       750000        1.0                  59.635   
                                 800000        1.0                  64.149   
                                 850000        1.0                  68.120   
                                 900000        1.0                  72.926   
                                 950000        1.0                  76.732   

                                         Runtime (seconds)  Valid  
Method            Description    Size                              
insertion_sort    reverse_sorted 50000            2.150072    1.0  
                                 100000           8.632831    1.0  
                                 150000          18.871199    1.0  
                                 200000          33.562805    1.0  
                                 250000          54.213657    1.0  
...                                                    ...    ...  
vanilla_quicksort unsorted       750000           0.059635    1.0  
                                 800000           0.064149    1.0  
                                 850000           0.068120    1.0  
                                 900000           0.072926    1.0  
                                 950000           0.076732    1.0  

[285 rows x 4 columns]